In [17]:
import re
import math
import xml.etree.ElementTree as ET
from collections import defaultdict, Counter
import tkinter as tk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

query_expansion_map = {
    "dbms": "database management system",
    "ai": "artificial intelligence",
    "ml": "machine learning",
    "nlp": "natural language processing",
    "introductory": "introduction fundamentals basics overview",
    "lab": "laboratory",
    "easy": "foundation beginner fundamentals" 
}


In [18]:
def classify_course(course_code):
    code_num = int(re.findall(r'\d+', course_code)[0])

    if 5000 <= code_num <= 5009:
        return "Master's Align"
    elif code_num < 5000:
        return "Undergraduate"
    return "Unclassified"

def is_easy_course(description):
    easy_keywords = ["foundation", "beginner", "fundamentals", "introductory", "basic"]
    description = description.lower()
    return any(keyword in description for keyword in easy_keywords)


In [19]:
query_relevance_map = {
    "which courses teach about artificial intelligence?": ["CS 4100", "CS 5100", "CS 5150", "CS 5170", "CS 7170", "CS 7180", "CS 4150"],
    "courses related to machine learning.": ["CS 6140", "CS 4170", "CS 5150", "CS 5100"],
    "show all introductory courses.": ["CS 1100", "CS 1101", "CS 2500", "CS 2510"],
    "which courses cover data structures?": ["CS 2500", "CS 2510", "CS 5010"],
    "what are some easy courses in ai?": ["CS 4050", "CS 5150"],
    "which courses are labs or include lab work?": ["CS 1101", "CS 4700"],
    "find a course that teaches databases or dbms.": ["CS 3200", "CS 5200"],
    "which course introduces programming in python?": ["CS 1100"],
    "courses that include natural language processing.": ["CS 6120", "CS 7120"],
    "what courses are advanced in ai and have high credit hours?": ["CS 5100", "CS 5170", "CS 7170"],
    "ai and advanced": ["CS 5100", "CS 5170", "CS 7170"],
    "machine and not learning": ["CS 4100"],
    "database or data or dbms": ["CS 3200", "CS 5200", "CS 6200"],
    "introductory and (ai or machine learning)": ["CS 1100", "CS 5100"],
    "programming and not (web or mobile)": ["CS 2500", "CS 5010"],
    "which undergraduate courses require cs 2500 as a prerequisite?": ["CS 2510", "CS 3200", "CS 3500", "CS 3700"],
    "show master's courses that have more than 3 credit hours.": ["CS 5100", "CS 6140", "CS 6200"],
    "which courses have no prerequisites?": ["CS 1100", "CS 1990"],
    "courses that list cs 1101 as a corequisite.": ["CS 1100"],
    "which 4-credit-hour courses are ai-related?": ["CS 4100", "CS 5100", "CS 5150", "CS 5170"],
    "master's align courses only.": ["CS 5001", "CS 5002", "CS 5010"],
    "courses with lab component and high hours.": ["CS 4700"],
    "undergrad courses with prerequisites.": ["CS 2500", "CS 2510", "CS 3200", "CS 3500"],
    "introductory database courses.": ["CS 3200"],
    "courses that are electives and ai-related.": ["CS 4050", "CS 4170"]
}


In [20]:
def preprocess_text(text):
    tokens = re.findall(r"\b\w+\b", text.lower())
    return [stemmer.stem(t) for t in tokens if t not in stop_words]


In [21]:
def build_index(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    inv_index = defaultdict(set)
    documents = {}
    meta = {}

    for course in root.findall("course"):
        code = course.find("code").text.strip()
        name = course.find("name").text.strip()
        desc = course.find("description").text.strip()
        hours = course.find("hours").text.strip()

        pre = course.find("prerequisites")
        co = course.find("corequisites")

        prereqs = [p.text.strip() for p in pre.findall("prerequisite")] if pre is not None else []
        coreqs = [c.text.strip() for c in co.findall("corequisite")] if co is not None else []

        tokens = preprocess_text(f"{name} {desc}")
        documents[code] = tokens
        meta[code] = {
            "name": name,
            "description": desc,
            "hours": int(hours) if hours.isdigit() else 0,
            "prerequisites": prereqs,
            "corequisites": coreqs
        }

        for token in tokens:
            inv_index[token].add(code)

    return inv_index, documents, meta


In [22]:
from collections import defaultdict

def build_prereq_graph(meta):
    graph = defaultdict(set)
    for course, data in meta.items():
        for prereq in data["prerequisites"]:
            if prereq in meta:
                graph[prereq].add(course)
    return graph


In [23]:
def compute_pagerank(graph, alpha=0.85, max_iter=100, tol=1e-6):
    nodes = set(graph.keys()) | {c for courses in graph.values() for c in courses}
    pr = {node: 1 / len(nodes) for node in nodes}

    for _ in range(max_iter):
        prev_pr = pr.copy()
        for node in nodes:
            incoming = [src for src in graph if node in graph[src]]
            pr[node] = (1 - alpha) / len(nodes) + alpha * sum(
                prev_pr[i] / len(graph[i]) for i in incoming if len(graph[i]) > 0
            )

        # Check convergence
        if sum(abs(pr[n] - prev_pr[n]) for n in nodes) < tol:
            break

    return pr


In [24]:
def rerank_with_pagerank(results, pagerank, weight=0.2):
    return sorted(results, key=lambda x: pagerank.get(x[0], 0) * weight, reverse=True)


In [25]:
def preprocess_query(query, expand=False):
    tokens = re.findall(r"\b\w+\b", query.lower())
    if expand:
        expanded = []
        for t in tokens:
            if t in query_expansion_map:
                expanded += query_expansion_map[t].split()
            else:
                expanded.append(t)
        tokens = expanded
    return [stemmer.stem(t) for t in tokens if t not in stop_words]

def process_boolean_query(tokens, inv_index):
    result = set()
    op = "OR"
    for token in tokens:
        if token.upper() in {"AND", "OR", "NOT"}:
            op = token.upper()
        else:
            docs = inv_index.get(token, set())
            if op == "OR":
                result |= docs
            elif op == "AND":
                result &= docs if result else docs
            elif op == "NOT":
                result -= docs
    return result


In [26]:
k1, b = 1.5, 0.75

def compute_bm25_scores(query_tokens, documents):
    N = len(documents)
    avg_dl = sum(len(doc) for doc in documents.values()) / N
    df = defaultdict(int)

    for doc in documents.values():
        for term in set(doc):
            df[term] += 1

    idf = {term: math.log((N - df[term] + 0.5) / (df[term] + 0.5) + 1) for term in query_tokens}
    scores = {}

    for doc_id, doc in documents.items():
        tf = Counter(doc)
        dl = len(doc)
        score = 0
        for term in query_tokens:
            if term in tf:
                f = tf[term]
                score += idf[term] * ((f * (k1 + 1)) / (f + k1 * (1 - b + b * dl / avg_dl)))
        if score > 0:
            scores[doc_id] = score

    return dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_tfidf_scores(query_tokens, documents):
    doc_ids = list(documents.keys())
    corpus = [' '.join(documents[doc_id]) for doc_id in doc_ids]
    query = ' '.join(query_tokens)

    # Vectorize with TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus + [query])

    # Compute cosine similarity between query and documents
    query_vec = tfidf_matrix[-1]
    doc_vecs = tfidf_matrix[:-1]
    similarities = cosine_similarity(query_vec, doc_vecs).flatten()

    # Return top-scoring documents
    scores = {doc_ids[i]: float(similarities[i]) for i in range(len(doc_ids)) if similarities[i] > 0}
    return dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))


In [28]:
def search(query, inv_index, documents, meta, use_bm25=True, use_tfidf=False, expand=True, pagerank=None, boost_weight=0.2):
    tokens = preprocess_query(query, expand=expand)
    query_lower = query.lower().strip()

    # Determine course level filters
    is_master = "master's" in query_lower and "align" not in query_lower
    is_align = "master's align" in query_lower or "align" in query_lower
    is_undergrad = "undergrad" in query_lower or "undergraduate" in query_lower

    # Step 1: Get initial candidate set
    if any(op in query.upper() for op in ("AND", "OR", "NOT")):
        candidates = process_boolean_query(tokens, inv_index)
    else:
        candidates = set()
        for token in tokens:
            candidates |= inv_index.get(token, set())

    # Step 2: Scoring
    if use_tfidf:
        scores = compute_tfidf_scores(tokens, documents)
    elif use_bm25:
        scores = compute_bm25_scores(tokens, documents)
    else:
        scores = {}

    # Step 3: Filter based on course level
    def filter_course(code):
        try:
            num = int(code.split()[1]) if code.startswith("CS ") else 0
            if is_master:
                return num >= 5000
            elif is_align:
                return 5000 <= num <= 5009
            elif is_undergrad:
                return num < 5000
            return True  # No filter if none specified
        except (ValueError, IndexError):
            return False  # Exclude malformed codes

    # Apply filter to candidates
    filtered_candidates = {code for code in candidates if filter_course(code)}

    # Step 4: Filter scored results and apply metadata
    results = [(code, meta[code]) for code in scores if code in filtered_candidates] if scores else [(code, meta[code]) for code in filtered_candidates]

    # Step 5: Apply PageRank boost
    if pagerank:
        for i, (code, data) in enumerate(results):
            data["score"] = scores.get(code, 0) + boost_weight * pagerank.get(code, 0)
        results = sorted(results, key=lambda x: x[1]["score"], reverse=True)
    elif scores:
        results = sorted(results, key=lambda x: scores.get(x[0], 0), reverse=True)

    return results

In [29]:
def precision_at_k(results, relevant, k=10):
    retrieved = [r[0] for r in results[:k]]
    return len(set(retrieved) & set(relevant)) / k

def ndcg_at_k(results, relevant, k=10):
    def dcg(rel):
        return sum([1 / math.log2(i + 2) if rel[i] else 0 for i in range(len(rel))])
    rels = [1 if r[0] in relevant else 0 for r in results[:k]]
    ideal = sorted(rels, reverse=True)
    return dcg(rels) / (dcg(ideal) or 1)


In [30]:
import tkinter as tk

def run_tk_gui(inv_index, docs, meta, pagerank_scores):
    def on_search():
        query = entry.get()
        result_display.config(state='normal')
        result_display.delete("1.0", tk.END)
        eval_label.config(text="")

        results = search(
            query,
            inv_index,
            docs,
            meta,
            use_bm25=not tfidf_var.get(),
            use_tfidf=tfidf_var.get(),
            pagerank=pagerank_scores if pagerank_var.get() else None
        )

        for i, (code, data) in enumerate(results[:10], 1):
            result_display.insert(tk.END, f"{i}. {code} — {data['name']}\n")
            result_display.insert(tk.END, f"   Description     : {data['description']}\n")
            result_display.insert(tk.END, f"   Prerequisite(s) : {', '.join(data['prerequisites']) or 'None'}\n")
            result_display.insert(tk.END, f"   Corequisite(s)  : {', '.join(data['corequisites']) or 'None'}\n")
            result_display.insert(tk.END, f"   Hours           : {data['hours']}\n")
            result_display.insert(tk.END, "-" * 100 + "\n")

        result_display.config(state='disabled')

        q_clean = query.lower().strip()
        if q_clean in query_relevance_map:
            relevant = query_relevance_map[q_clean]
            p10 = precision_at_k(results, relevant)
            ndcg = ndcg_at_k(results, relevant)
            eval_label.config(
                text=f"📊 Evaluation for '{q_clean}':  Precision@10 = {p10:.2f},  nDCG@10 = {ndcg:.2f}"
            )
        else:
            eval_label.config(text="📊 No relevance data for this query.")

    root = tk.Tk()
    root.title("Course Search Engine")

    tk.Label(root, text="Search Query:").pack()
    entry = tk.Entry(root, width=100)
    entry.pack(pady=4)

    tfidf_var = tk.BooleanVar()
    pagerank_var = tk.BooleanVar()
    tk.Checkbutton(root, text="Use TF-IDF instead of BM25", variable=tfidf_var).pack()
    tk.Checkbutton(root, text="Boost with PageRank", variable=pagerank_var).pack()

    tk.Button(root, text="Search", command=on_search).pack(pady=4)

    result_display = tk.Text(root, width=200, height=60, wrap="word")
    result_display.pack(padx=5, pady=5)
    result_display.config(state='disabled')

    eval_label = tk.Label(root, text="", font=("Helvetica", 16), fg="green")
    eval_label.pack(pady=5)

    root.mainloop()


In [31]:
def cli_search_interface(inv_index, docs, meta):
    print("=== Course Search Engine (CLI) ===")
    print("Type 'exit' to quit.\n")
    while True:
        query = input("Search Query: ")
        if query.lower() == "exit":
            break

        results = search(query, inv_index, docs, meta)

        print("\nTop Results:")
        for i, (code, data) in enumerate(results[:10], 1):
            print(f"{i}.")
            print(f"Course code     : {code}")
            print(f"Course name     : {data['name']}")
            print(f"Description     : {data['description']}")
            print(f"Prerequisite(s) : {', '.join(data['prerequisites']) or 'None'}")
            print(f"Corequisite(s)  : {', '.join(data['corequisites']) or 'None'}")
            print(f"Hours           : {data['hours']}")
            print("-" * 40)

        q_clean = query.lower().strip()
        if q_clean in query_relevance_map:
            relevant = query_relevance_map[q_clean]
            p10 = precision_at_k(results, relevant)
            ndcg = ndcg_at_k(results, relevant)
            print(f"📊 Evaluation for '{q_clean}':  Precision@10 = {p10:.2f},  nDCG@10 = {ndcg:.2f}")
        else:
            print("📊 No relevance data for this query.\n")


In [ ]:
if __name__ == "__main__":
    xml_file = "cs_courses.xml"  # 🔁 make sure this file exists
    inv_index, docs, meta = build_index(xml_file)
    prereq_graph = build_prereq_graph(meta)
    pagerank_scores = compute_pagerank(prereq_graph)

    mode = input("Choose interface [cli/gui]: ").lower()
    if mode == "cli":
        cli_search_interface(inv_index, docs, meta)
    elif mode == "gui":
        run_tk_gui(inv_index, docs, meta, pagerank_scores)